# Varo ML Models - Feature Store with Online Serving & Model Registry

This notebook implements a complete Snowflake Feature Store with **online feature serving** for real-time inference, followed by ML model training.

## Features
- **Online Feature Serving**: Sub-second latency for real-time inference
- **4 Feature Views**: CUSTOMER_PROFILE, TRANSACTION_PATTERN, ADVANCE_RISK, FRAUD_DETECTION
- **3 ML Models**: Fraud Detection, Advance Eligibility, Customer LTV
- **Incremental Refresh**: Efficient data sync using change tracking

## Prerequisites
- Snowflake v9.26+ and snowflake-ml-python v1.18.0+
- Database: VARO_INTELLIGENCE
- Warehouse: VARO_FEATURE_WH

## 1. Import Packages

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Feature Store with Online Config
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode,
    StoreType
)
from snowflake.ml.feature_store.feature_view import OnlineConfig

# ML Pipeline & Registry
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier, GradientBoostingRegressor
from snowflake.ml.modeling.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from snowflake.ml.registry import Registry

print("✅ All packages imported successfully")

## 2. Connect to Snowflake

In [ ]:
session = get_active_session()
session.use_database('VARO_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('VARO_FEATURE_WH')

print(f"✅ Connected to Snowflake")
print(f"   Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Context: {session.get_fully_qualified_current_schema()}")

## 3. Setup Feature Store & Entity

In [ ]:
# Connect to Feature Store
fs = FeatureStore(
    session=session,
    database="VARO_INTELLIGENCE",
    name="FEATURE_STORE",
    default_warehouse="VARO_FEATURE_WH",
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

# Register CUSTOMER entity
customer_entity = Entity(
    name="CUSTOMER",
    join_keys=["customer_id"],
    desc="Varo bank customer"
)
fs.register_entity(customer_entity)

print("✅ Feature Store connected")
print("✅ CUSTOMER entity registered")

## 4. One-Time Setup: Enable Change Tracking & Clean Previous Failures

In [ ]:
# Step 1: Create aggregation views (required for Feature Views)
aggregation_views_sql = """
-- Customer Profile Aggregations
CREATE OR REPLACE VIEW VARO_INTELLIGENCE.FEATURE_STORE.V_CUSTOMER_PROFILE_AGGS AS
SELECT
    c.customer_id,
    COUNT(DISTINCT a.account_id) as num_accounts
FROM VARO_INTELLIGENCE.RAW.CUSTOMERS c
LEFT JOIN VARO_INTELLIGENCE.RAW.ACCOUNTS a ON c.customer_id = a.customer_id
GROUP BY c.customer_id;

-- Transaction Pattern Aggregations
CREATE OR REPLACE VIEW VARO_INTELLIGENCE.FEATURE_STORE.V_TRANSACTION_PATTERN_AGGS AS
SELECT
    customer_id,
    COUNT(CASE WHEN transaction_date >= DATEADD('day', -7, CURRENT_DATE()) THEN 1 END) as txn_count_7d,
    SUM(CASE WHEN transaction_date >= DATEADD('day', -7, CURRENT_DATE()) THEN ABS(amount) END) as txn_volume_7d,
    COUNT(CASE WHEN transaction_timestamp >= DATEADD('hour', -1, CURRENT_TIMESTAMP()) THEN 1 END) as txn_count_1h
FROM VARO_INTELLIGENCE.RAW.TRANSACTIONS
WHERE status = 'COMPLETED'
GROUP BY customer_id;

-- Advance Risk Aggregations
CREATE OR REPLACE VIEW VARO_INTELLIGENCE.FEATURE_STORE.V_ADVANCE_RISK_AGGS AS
SELECT
    c.customer_id,
    COUNT(ca.advance_id) as total_advances_taken,
    AVG(ca.advance_amount) as avg_advance_amount,
    COUNT(CASE WHEN ca.advance_status = 'DEFAULTED' THEN 1 END) as num_defaults
FROM VARO_INTELLIGENCE.RAW.CUSTOMERS c
LEFT JOIN VARO_INTELLIGENCE.RAW.CASH_ADVANCES ca ON c.customer_id = ca.customer_id
GROUP BY c.customer_id;

-- Fraud Detection Aggregations
CREATE OR REPLACE VIEW VARO_INTELLIGENCE.FEATURE_STORE.V_FRAUD_DETECTION_AGGS AS
WITH recent_transactions AS (
    SELECT 
        t.*,
        LAG(transaction_timestamp) OVER (PARTITION BY customer_id ORDER BY transaction_timestamp) as prev_txn_time,
        LAG(merchant_city) OVER (PARTITION BY customer_id ORDER BY transaction_timestamp) as prev_merchant_city
    FROM VARO_INTELLIGENCE.RAW.TRANSACTIONS t
    WHERE transaction_timestamp >= DATEADD('day', -7, CURRENT_TIMESTAMP()) AND status = 'COMPLETED'
)
SELECT
    t.customer_id,
    MAX(CASE WHEN ABS(t.amount) > avg_txn.avg_amount * 3 AND ABS(t.amount) > 100 THEN 1 ELSE 0 END) as has_unusual_amount,
    MAX(CASE WHEN DATEDIFF('minute', t.prev_txn_time, t.transaction_timestamp) < 5 AND t.merchant_city != t.prev_merchant_city THEN 1 ELSE 0 END) as impossible_travel_flag
FROM recent_transactions t
LEFT JOIN (
    SELECT customer_id, AVG(ABS(amount)) as avg_amount
    FROM VARO_INTELLIGENCE.RAW.TRANSACTIONS
    WHERE transaction_date >= DATEADD('day', -30, CURRENT_DATE())
    GROUP BY customer_id
) avg_txn ON t.customer_id = avg_txn.customer_id
GROUP BY t.customer_id;
"""

for command in aggregation_views_sql.split(';'):
    if command.strip():
        try:
            session.sql(command).collect()
        except Exception as e:
            if "already exists" not in str(e):
                raise e
print("✅ Aggregation views created")

# Step 2: Clean up any corrupted objects from previous failed registrations
view_names = [
    '"CUSTOMER_PROFILE_FEATURES$v1"',
    '"TRANSACTION_PATTERN_FEATURES$v1"',
    '"ADVANCE_RISK_FEATURES$v1"',
    '"FRAUD_DETECTION_FEATURES$v1"'
]

for view_name in view_names:
    try:
        session.sql(f'DROP DYNAMIC TABLE IF EXISTS VARO_INTELLIGENCE.FEATURE_STORE.{view_name}').collect()
    except:
        pass

print("✅ Cleaned up previous registration attempts")
print("\n🔧 Setup complete - ready to register Feature Views")
print("   Note: Using FULL refresh mode (no change tracking required)")

## 5. Configure Online Feature Serving

In [ ]:
# Real-time serving: 30-second freshness (for fraud detection)
online_config_realtime = OnlineConfig(
    enable=True,
    target_lag="30 seconds"
)

# Hourly serving: 5-minute freshness (for profile features)
online_config_hourly = OnlineConfig(
    enable=True,
    target_lag="5 minutes"
)

print("✅ Online configs defined")
print("   - Realtime: 30 seconds")
print("   - Hourly: 5 minutes")

# CRITICAL: Switch to FEATURE_STORE schema before registering Feature Views
# This ensures the online table creation can find the offline Dynamic Table
session.use_schema('FEATURE_STORE')
print("✅ Switched to FEATURE_STORE schema for registration")

## 6. Register Feature Views with Online Serving

In [ ]:
# 1. CUSTOMER_PROFILE_FEATURES - Query ONLY the source table, no JOINs
customer_profile_df = session.sql("""
    SELECT
        customer_id,
        updated_at as feature_timestamp,
        date_of_birth,
        acquisition_date,
        credit_score,
        income_verified
    FROM VARO_INTELLIGENCE.RAW.CUSTOMERS
    WHERE customer_status = 'ACTIVE'
""")

customer_profile_fv = FeatureView(
    name="CUSTOMER_PROFILE_FEATURES",
    entities=[customer_entity],
    feature_df=customer_profile_df,
    timestamp_col="feature_timestamp",
    refresh_freq="1 hour",
    refresh_mode="FULL",
    desc="Customer demographic, account, and profile features",
    online_config=online_config_hourly
)
fs.register_feature_view(feature_view=customer_profile_fv, version="v1", block=True)
print("✅ CUSTOMER_PROFILE_FEATURES registered with online serving (5min lag)")

# 2. TRANSACTION_PATTERN_FEATURES
transaction_pattern_df = session.sql("""
    SELECT DISTINCT
        t.customer_id,
        t.created_at as feature_timestamp,
        COALESCE(v.txn_count_7d, 0) as txn_count_7d,
        COALESCE(v.txn_volume_7d, 0) as txn_volume_7d,
        COALESCE(v.txn_count_1h, 0) as txn_count_1h
    FROM VARO_INTELLIGENCE.RAW.TRANSACTIONS t
    LEFT JOIN VARO_INTELLIGENCE.FEATURE_STORE.V_TRANSACTION_PATTERN_AGGS v ON t.customer_id = v.customer_id
    WHERE t.status = 'COMPLETED'
""")

transaction_pattern_fv = FeatureView(
    name="TRANSACTION_PATTERN_FEATURES",
    entities=[customer_entity],
    feature_df=transaction_pattern_df,
    timestamp_col="feature_timestamp",
    refresh_freq="30 minutes",
    refresh_mode="FULL",
    desc="Transaction patterns, velocity, and spending behavior",
    online_config=online_config_realtime
)
fs.register_feature_view(feature_view=transaction_pattern_fv, version="v1", block=True)
print("✅ TRANSACTION_PATTERN_FEATURES registered with online serving (30sec lag)")

# 3. ADVANCE_RISK_FEATURES
advance_risk_df = session.sql("""
    SELECT
        c.customer_id,
        c.updated_at as feature_timestamp,
        COALESCE(v.total_advances_taken, 0) as total_advances_taken,
        COALESCE(v.avg_advance_amount, 0) as avg_advance_amount,
        COALESCE(v.num_defaults, 0) as num_defaults
    FROM VARO_INTELLIGENCE.RAW.CUSTOMERS c
    LEFT JOIN VARO_INTELLIGENCE.FEATURE_STORE.V_ADVANCE_RISK_AGGS v ON c.customer_id = v.customer_id
    WHERE c.customer_status = 'ACTIVE'
""")

advance_risk_fv = FeatureView(
    name="ADVANCE_RISK_FEATURES",
    entities=[customer_entity],
    feature_df=advance_risk_df,
    timestamp_col="feature_timestamp",
    refresh_freq="1 hour",
    refresh_mode="FULL",
    desc="Cash advance history and risk indicators",
    online_config=online_config_hourly
)
fs.register_feature_view(feature_view=advance_risk_fv, version="v1", block=True)
print("✅ ADVANCE_RISK_FEATURES registered with online serving (5min lag)")

# 4. FRAUD_DETECTION_FEATURES
fraud_detection_df = session.sql("""
    SELECT DISTINCT
        t.customer_id,
        t.created_at as feature_timestamp,
        COALESCE(v.has_unusual_amount, 0) as has_unusual_amount,
        COALESCE(v.impossible_travel_flag, 0) as impossible_travel_flag
    FROM VARO_INTELLIGENCE.RAW.TRANSACTIONS t
    LEFT JOIN VARO_INTELLIGENCE.FEATURE_STORE.V_FRAUD_DETECTION_AGGS v ON t.customer_id = v.customer_id
    WHERE t.status = 'COMPLETED'
""")

fraud_detection_fv = FeatureView(
    name="FRAUD_DETECTION_FEATURES",
    entities=[customer_entity],
    feature_df=fraud_detection_df,
    timestamp_col="feature_timestamp",
    refresh_freq="15 minutes",
    refresh_mode="FULL",
    desc="Real-time fraud indicators and anomalies",
    online_config=online_config_realtime
)
fs.register_feature_view(feature_view=fraud_detection_fv, version="v1", block=True)
print("✅ FRAUD_DETECTION_FEATURES registered with online serving (30sec lag)")

print("\n🎉 All Feature Views registered with online serving enabled!")
print("   Access via Snowflake UI: AI/ML > Features")

## 7. Demonstrate Online Feature Retrieval (Low-Latency Inference)

In [ ]:
# Get a sample customer ID
sample_cust = session.sql("SELECT customer_id FROM VARO_INTELLIGENCE.RAW.CUSTOMERS LIMIT 1").collect()
if sample_cust:
    customer_id = sample_cust[0][0]
    
    # Retrieve features from ONLINE store (sub-second latency)
    try:
        online_features = fs.read_feature_view(
            feature_view=fraud_detection_fv,
            keys=[[customer_id]],
            store_type=StoreType.ONLINE
        )
        print(f"✅ Retrieved online features for customer {customer_id}")
        print(f"   Latency: Sub-second from online store")
        print(online_features.show())
    except Exception as e:
        print(f"ℹ️  Online store still warming up: {e}")
        print("   This is normal on first run - data syncs in background")
else:
    print("ℹ️  No customers found - ensure data is loaded")

---
# ML MODEL TRAINING

## 8. Model 1: Transaction Fraud Detection

In [ ]:
# Load fraud training data
fraud_df = session.sql("""
SELECT
    t.transaction_id,
    t.customer_id,
    t.amount::FLOAT AS amount,
    COALESCE(t.merchant_category, 'UNKNOWN') as merchant_category,
    COALESCE(t.transaction_type, 'DEBIT') as transaction_type,
    COALESCE(t.is_international, FALSE)::BOOLEAN AS is_international,
    COALESCE(c.credit_score, 650)::FLOAT AS credit_score,
    COALESCE(c.risk_tier, 'MEDIUM') as risk_tier,
    COALESCE(a.current_balance, 0)::FLOAT AS account_balance,
    (COALESCE(t.fraud_score, 0) > 0.7)::BOOLEAN AS is_fraud
FROM VARO_INTELLIGENCE.RAW.TRANSACTIONS t
LEFT JOIN VARO_INTELLIGENCE.RAW.CUSTOMERS c ON t.customer_id = c.customer_id
LEFT JOIN VARO_INTELLIGENCE.RAW.ACCOUNTS a ON t.account_id = a.account_id
WHERE t.transaction_date >= DATEADD('month', -6, CURRENT_DATE())
  AND t.amount > 10
LIMIT 10000
""")

print(f"✅ Fraud dataset: {fraud_df.count()} transactions")

In [ ]:
# Split and prepare
train_fraud, test_fraud = fraud_df.random_split([0.8, 0.2], seed=42)
train_fraud = train_fraud.drop("TRANSACTION_ID", "CUSTOMER_ID").fillna({"ACCOUNT_BALANCE": 0, "CREDIT_SCORE": 650.0})
test_fraud = test_fraud.drop("TRANSACTION_ID", "CUSTOMER_ID").fillna({"ACCOUNT_BALANCE": 0, "CREDIT_SCORE": 650.0})

# Pipeline
fraud_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["MERCHANT_CATEGORY", "TRANSACTION_TYPE", "RISK_TIER"],
        output_cols=["MERCHANT_CATEGORY_ENC", "TRANSACTION_TYPE_ENC", "RISK_TIER_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Scaler", StandardScaler(
        input_cols=["AMOUNT", "CREDIT_SCORE", "ACCOUNT_BALANCE"],
        output_cols=["AMOUNT_SCALED", "CREDIT_SCORE_SCALED", "ACCOUNT_BALANCE_SCALED"]
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["IS_FRAUD"],
        output_cols=["FRAUD_PREDICTION"],
        n_estimators=100,
        max_depth=10
    ))
])

fraud_pipeline.fit(train_fraud)
print("✅ Fraud model trained")

In [ ]:
# Evaluate
fraud_predictions = fraud_pipeline.predict(test_fraud)
fraud_accuracy = accuracy_score(df=fraud_predictions, y_true_col_names="IS_FRAUD", y_pred_col_names="FRAUD_PREDICTION")

# Register
reg = Registry(session)
fraud_version = reg.log_model(
    model=fraud_pipeline,
    model_name="FRAUD_DETECTION_MODEL",
    comment="Random Forest fraud classifier",
    metrics={"accuracy": round(fraud_accuracy, 4)}
)

print(f"✅ FRAUD_DETECTION_MODEL registered - Accuracy: {fraud_accuracy:.4f}")

## 9. Model 2: Cash Advance Repayment Success

In [ ]:
# Load advance training data
advance_df = session.sql("""
SELECT
    ca.advance_id,
    ca.customer_id,
    ca.advance_amount::FLOAT AS advance_amount,
    ca.fee_amount::FLOAT AS fee_amount,
    ca.eligibility_score::FLOAT AS eligibility_score,
    c.credit_score::FLOAT AS credit_score,
    c.risk_tier,
    c.employment_status,
    COUNT(DISTINCT dd.deposit_id)::FLOAT AS deposit_count,
    AVG(dd.amount)::FLOAT AS avg_deposit_amount,
    (ca.advance_status = 'REPAID')::BOOLEAN AS was_repaid
FROM VARO_INTELLIGENCE.RAW.CASH_ADVANCES ca
INNER JOIN VARO_INTELLIGENCE.RAW.CUSTOMERS c ON ca.customer_id = c.customer_id
INNER JOIN VARO_INTELLIGENCE.RAW.DIRECT_DEPOSITS dd ON ca.customer_id = dd.customer_id
WHERE ca.advance_date >= DATEADD('month', -12, CURRENT_DATE())
  AND ca.eligibility_score IS NOT NULL
  AND c.credit_score IS NOT NULL
  AND c.risk_tier IS NOT NULL
  AND c.employment_status IS NOT NULL
  AND dd.amount IS NOT NULL
GROUP BY ca.advance_id, ca.customer_id, ca.advance_amount, ca.fee_amount, ca.eligibility_score,
         c.credit_score, c.risk_tier, c.employment_status, ca.advance_status
HAVING AVG(dd.amount) IS NOT NULL AND COUNT(DISTINCT dd.deposit_id) > 0
LIMIT 5000
""")

print(f"✅ Advance dataset: {advance_df.count()} records")

In [ ]:
# Split and prepare
train_adv, test_adv = advance_df.random_split([0.8, 0.2], seed=42)
train_adv = train_adv.drop("ADVANCE_ID", "CUSTOMER_ID").fillna({
    "ADVANCE_AMOUNT": 100.0, "FEE_AMOUNT": 5.0, "ELIGIBILITY_SCORE": 0.5,
    "CREDIT_SCORE": 650.0, "DEPOSIT_COUNT": 0.0, "AVG_DEPOSIT_AMOUNT": 1000.0
})
test_adv = test_adv.drop("ADVANCE_ID", "CUSTOMER_ID").fillna({
    "ADVANCE_AMOUNT": 100.0, "FEE_AMOUNT": 5.0, "ELIGIBILITY_SCORE": 0.5,
    "CREDIT_SCORE": 650.0, "DEPOSIT_COUNT": 0.0, "AVG_DEPOSIT_AMOUNT": 1000.0
})

# Pipeline
advance_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["RISK_TIER", "EMPLOYMENT_STATUS"],
        output_cols=["RISK_TIER_ENC", "EMPLOYMENT_STATUS_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Scaler", StandardScaler(
        input_cols=["ADVANCE_AMOUNT", "FEE_AMOUNT", "ELIGIBILITY_SCORE", "CREDIT_SCORE", "DEPOSIT_COUNT"],
        output_cols=["ADVANCE_AMOUNT_SCALED", "FEE_AMOUNT_SCALED", "ELIGIBILITY_SCORE_SCALED", "CREDIT_SCORE_SCALED", "DEPOSIT_COUNT_SCALED"]
    )),
    ("Classifier", LogisticRegression(
        label_cols=["WAS_REPAID"],
        output_cols=["REPAYMENT_PREDICTION"]
    ))
])

advance_pipeline.fit(train_adv)
print("✅ Advance model trained")

In [ ]:
# Evaluate
advance_predictions = advance_pipeline.predict(test_adv)
advance_accuracy = accuracy_score(df=advance_predictions, y_true_col_names="WAS_REPAID", y_pred_col_names="REPAYMENT_PREDICTION")

# Register
advance_version = reg.log_model(
    model=advance_pipeline,
    model_name="ADVANCE_ELIGIBILITY_MODEL",
    comment="Logistic Regression for advance repayment prediction",
    metrics={"accuracy": round(advance_accuracy, 4)}
)

print(f"✅ ADVANCE_ELIGIBILITY_MODEL registered - Accuracy: {advance_accuracy:.4f}")

## 10. Model 3: Customer Lifetime Value Prediction

In [ ]:
# Load LTV training data
ltv_df = session.sql("""
SELECT
    c.customer_id,
    c.lifetime_value::FLOAT AS lifetime_value,
    DATEDIFF('month', c.acquisition_date, CURRENT_DATE())::FLOAT AS tenure_months,
    c.credit_score::FLOAT AS credit_score,
    c.risk_tier,
    c.acquisition_channel,
    COUNT(DISTINCT a.account_id)::FLOAT AS product_count,
    COALESCE(AVG(a.current_balance), 0)::FLOAT AS avg_account_balance,
    COUNT(DISTINCT CASE WHEN t.transaction_date >= DATEADD('day', -90, CURRENT_DATE())
                   THEN t.transaction_id END)::FLOAT AS recent_transaction_count,
    (COUNT(DISTINCT dd.deposit_id) > 0)::BOOLEAN AS has_direct_deposit
FROM VARO_INTELLIGENCE.RAW.CUSTOMERS c
LEFT JOIN VARO_INTELLIGENCE.RAW.ACCOUNTS a ON c.customer_id = a.customer_id
LEFT JOIN VARO_INTELLIGENCE.RAW.TRANSACTIONS t ON c.customer_id = t.customer_id
LEFT JOIN VARO_INTELLIGENCE.RAW.DIRECT_DEPOSITS dd ON c.customer_id = dd.customer_id
WHERE c.customer_status = 'ACTIVE' AND c.lifetime_value > 0
GROUP BY c.customer_id, c.lifetime_value, c.acquisition_date, c.credit_score, c.risk_tier, c.acquisition_channel
LIMIT 5000
""")

print(f"✅ LTV dataset: {ltv_df.count()} customers")

In [ ]:
# Split and prepare
train_ltv, test_ltv = ltv_df.random_split([0.8, 0.2], seed=42)
train_ltv = train_ltv.drop("CUSTOMER_ID").fillna({
    "TENURE_MONTHS": 1.0, "CREDIT_SCORE": 650.0, "PRODUCT_COUNT": 1.0,
    "AVG_ACCOUNT_BALANCE": 0.0, "RECENT_TRANSACTION_COUNT": 0.0
})
test_ltv = test_ltv.drop("CUSTOMER_ID").fillna({
    "TENURE_MONTHS": 1.0, "CREDIT_SCORE": 650.0, "PRODUCT_COUNT": 1.0,
    "AVG_ACCOUNT_BALANCE": 0.0, "RECENT_TRANSACTION_COUNT": 0.0
})

# Pipeline
ltv_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["RISK_TIER", "ACQUISITION_CHANNEL"],
        output_cols=["RISK_TIER_ENC", "ACQUISITION_CHANNEL_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Scaler", StandardScaler(
        input_cols=["TENURE_MONTHS", "CREDIT_SCORE", "PRODUCT_COUNT", "AVG_ACCOUNT_BALANCE", "RECENT_TRANSACTION_COUNT"],
        output_cols=["TENURE_MONTHS_SCALED", "CREDIT_SCORE_SCALED", "PRODUCT_COUNT_SCALED", "AVG_ACCOUNT_BALANCE_SCALED", "RECENT_TRANSACTION_COUNT_SCALED"]
    )),
    ("Regressor", GradientBoostingRegressor(
        label_cols=["LIFETIME_VALUE"],
        output_cols=["PREDICTED_LTV"],
        n_estimators=100,
        max_depth=6
    ))
])

ltv_pipeline.fit(train_ltv)
print("✅ LTV model trained")

In [ ]:
# Evaluate
ltv_predictions = ltv_pipeline.predict(test_ltv)
ltv_mae = mean_absolute_error(df=ltv_predictions, y_true_col_names="LIFETIME_VALUE", y_pred_col_names="PREDICTED_LTV")
ltv_rmse = mean_squared_error(df=ltv_predictions, y_true_col_names="LIFETIME_VALUE", y_pred_col_names="PREDICTED_LTV") ** 0.5

# Register
ltv_version = reg.log_model(
    model=ltv_pipeline,
    model_name="CUSTOMER_LTV_MODEL",
    comment="Gradient Boosting for LTV prediction",
    metrics={"mae": round(ltv_mae, 2), "rmse": round(ltv_rmse, 2)}
)

print(f"✅ CUSTOMER_LTV_MODEL registered - MAE: {ltv_mae:.2f}, RMSE: {ltv_rmse:.2f}")

## 11. Summary: All Models & Feature Views Registered

In [ ]:
print("\n" + "="*70)
print("🎉 DEPLOYMENT COMPLETE")
print("="*70)

print("\n📊 FEATURE VIEWS (with Online Serving):")
print("  ✅ CUSTOMER_PROFILE_FEATURES (5-minute freshness)")
print("  ✅ TRANSACTION_PATTERN_FEATURES (30-second freshness)")
print("  ✅ ADVANCE_RISK_FEATURES (5-minute freshness)")
print("  ✅ FRAUD_DETECTION_FEATURES (30-second freshness)")

print("\n🤖 ML MODELS (in Registry):")
print("  ✅ FRAUD_DETECTION_MODEL")
print("  ✅ ADVANCE_ELIGIBILITY_MODEL")
print("  ✅ CUSTOMER_LTV_MODEL")

print("\n🚀 NEXT STEPS:")
print("  1. Features visible in Snowflake UI: AI/ML > Features")
print("  2. Models visible in Snowflake UI: AI/ML > Models")
print("  3. Online serving enables sub-second latency inference")
print("  4. Run sql/agent/10_create_intelligence_agent.sql to add to agent")
print("\n" + "="*70)